In [ ]:
# notebook setup
import datetime as dt
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))

import brightenv2.utils.gps as gps

InteractiveShell.ast_node_interactivity = 'all'

# bottom left, top right of the greater Seattle area
min_lat, min_lon = 47.324739, -122.440531
max_lat, max_lon = 48.100469, -121.850196

place_id = 'ChIJQ6ZdDwY4tokRfTVsHLUigJI'

### All `gps_records` as a parameter should be a list of `gps.GPS` which is a Python `namedtuple`.

In [ ]:
gps_records[:5]

### Impute stationary GPS records at a specific time interval

In [ ]:
now = dt.datetime.now()
now = dt.datetime(year=now.year, month=now.month, day=now.day, hour=12)

gps_records_ = [
    gps.GPS(lat=0, lon=0, ts=now+dt.timedelta(hours=i))
    for i in range(3)
]

pd.DataFrame(gps_records_).head()

gps_records_1 = gps.impute_stationary_coordinates(gps_records_)
pd.DataFrame(gps_records_1)

gps_records_2 = gps.impute_stationary_coordinates(gps_records_, freq='30Min')
pd.DataFrame(gps_records_2)

### Determine semantic velocity

In [ ]:
import brightenv2.utils.gps as gps

now, td = dt.datetime.now(), dt.timedelta(minutes=1)
lat, lon = -33.8666199, 151.1958527

a, b = (lat, lon, now), (lat + 0.0001, lon + 0.0001, now+td)
gps.discrete_velocity(a, b)

a, b = (lat, lon, now), (lat + 0.001, lon + 0.001, now+td)
gps.discrete_velocity(a, b)

a, b = (lat, lon, now), (lat + 0.01, lon + 0.01, now+td)
gps.discrete_velocity(a, b)

a, b = (lat, lon, now), (lat + 0.1, lon + 0.1, now+td)
gps.discrete_velocity(a, b)

a, b = (lat, lon, now), (lat + 1, lon + 1, now+td)
gps.discrete_velocity(a, b)

### Estimate home and work locations from a sequence of GPS points

In [ ]:
gps.estimate_home_location(gps_records)[0]
gps.estimate_work_location(gps_records)[0]

### Calculate the geographic distance between two coordinates

In [ ]:
a, b = (-45, 122), (-45, 100)
gps.geo_distance(*a, *b)

### Find clusters from a sequence of GPS points

In [ ]:
# a basic request with default parameters
labels, clusters = gps.gps_dbscan(gps_records)
pd.DataFrame(clusters).head()

In [ ]:
# an advance request with optional scikit-learn parameters
parameters = {
    'eps': 0.001,
    'min_samples': 4,
    'metric': 'euclidean',
    'n_jobs': -1
}

labels, clusters = gps.gps_dbscan(gps_records, parameters=parameters)
pd.DataFrame(clusters).head()

### Find clusters and add references as columns in an existing dataframe of gps records

In [ ]:
records, clusters = gps.get_clusters_with_context(pd.DataFrame(gps_records))

records.loc[records.cid != '-1'].head()
clusters.head()

### Find the times a subject entered and exited a cluster

In [ ]:
cluster_entries = gps.get_cluster_times(clusters, records)
cluster_entries.head()

### Calculate basic cluster metrics

### Find nearby places

In [ ]:
# lat, lon = clusters[0].get('lat'), clusters[0].get('lon')
# nearby = gps.request_nearby_places(lat, lon); nearby[0].as_dict